# Benchmarking Results for Flower Federated Learning Framework 🌸

## General Experiment Configuration
- **Federation Type**: Local Simulation
- **Dataset**: Fashion-MNIST (With Non-IID partitioning using Dirichlet distribution, α=0.3)
- **Model**: Simple CNN
- **Learning Rate**: 0.01
- **Flower Version**: 1.22.0
- **Python Version**: 3.10.18  

### Experiment Specific Configuration
- **Number of SuperNodes**: 50 (Clients, analyzing only clients who participated in all rounds)
- **Number of Server Rounds**: 10
- **Fraction of Clients per Round**: 0.75
- **Fraction of Clients for Evaluation**: 0.75
- **Local Epochs**: 5
- **Log file**: `EXP_CNN_fashion_mnist_dataset_1_logs.json`

In [382]:
import json
import pandas as pd

# Load the JSON file
with open("EXP_CNN_fashion_mnist_dataset_1_logs.json", "r") as f:
    data = json.load(f)

#### Round-level Statistics
The following table summarizes the round-level statistics, including average duration, accuracy, and data processed per round.

In [383]:
# Prepare round-level data
round_data = []
for round_entry in data:
    clients = round_entry["clients_logs"]
    round_number = clients[0]["server_round_number"]
    total_examples = sum(c["num-examples"] for c in clients)
    avg_duration = sum(c["round_duration"] for c in clients) / len(clients)
    round_acc = round_entry.get("round_acc", None)
    total_data = round_entry.get("total_amount_data_round_mb", None)
    num_rounds = clients[0]["num_rounds"]
    lr = clients[0]["lr"]

    round_data.append({
        "round": round_number,
        "total_examples": total_examples,
        "avg_duration": avg_duration,
        "accuracy": round_acc,
        "data_mb": total_data,
    })

df_rounds = pd.DataFrame(round_data)
df_rounds

,round,total_examples,avg_duration,accuracy,data_mb
0,1,35066,0.457403,14.82,6.2738
1,2,33697,0.433378,64.97,6.2738
2,3,35359,0.445654,78.87,6.2738
3,4,37272,0.496218,81.86,6.2738
4,5,33870,0.430839,82.37,6.2738
5,6,35130,0.439586,83.62,6.2738
6,7,36411,0.477300,83.99,6.2738
7,8,35323,0.443118,83.32,6.2738
8,9,33836,0.431253,84.21,6.2738
9,10,35051,0.448954,85.17,6.2738


In [384]:
import plotly.express as px
fig2 = px.line(
    df_rounds, x="round", y="avg_duration",
    title="Average Round Duration",
    hover_data=["round", "avg_duration"]
)
fig2.show()

In [385]:
fig3 = px.line(
    df_rounds, x="round", y="accuracy",
    title="Accuracy per Round",
    hover_data=["round", "accuracy"]
)
fig3.show()

#### Rounds, Number of Examples, and Loss per Client Statistics

In [386]:
# Flatten the nested structure
records = []
for round_entry in data:
    round_number = round_entry.get("clients_logs", [])[0]["server_round_number"]
    for log in round_entry["clients_logs"]:
        records.append({
            "client_id": log["client_id"],
            "server_round_number": log["server_round_number"],
            "round_duration": log["round_duration"],
            "num_examples": log["num-examples"],
            "round_loss": log["round_loss"]
        })

In [387]:
df = pd.DataFrame(records)

In [388]:
# 1️⃣ Clients × Round Durations
df_duration = df.pivot(index="client_id", columns="server_round_number", values="round_duration")
df_duration = df_duration.round(2)
df_duration.head()

server_round_number,1,2,3,4,5,6,7,8,9,10
client_id,,,,,,,,,,
0,0.54,0.52,0.55,0.55,0.51,0.52,0.53,0.53,0.59,0.53
1,0.28,0.32,0.30,NaN,0.31,NaN,NaN,0.31,0.32,0.31
2,0.22,0.22,0.23,NaN,0.23,0.23,0.23,0.22,NaN,0.24
3,NaN,0.43,NaN,NaN,0.42,0.41,0.44,0.43,0.41,0.51
4,0.27,0.36,0.28,0.28,NaN,0.32,NaN,0.30,0.28,0.29


In [389]:
# 2️⃣ Clients × Num Examples
df_examples = df.pivot(index="client_id", columns="server_round_number", values="num_examples")
df_examples.head()

server_round_number,1,2,3,4,5,6,7,8,9,10
client_id,,,,,,,,,,
0,1131.0,1131.0,1131.0,1131.0,1131.0,1131.0,1131.0,1131.0,1131.0,1131.0
1,664.0,664.0,664.0,NaN,664.0,NaN,NaN,664.0,664.0,664.0
2,483.0,483.0,483.0,NaN,483.0,483.0,483.0,483.0,NaN,483.0
3,NaN,898.0,NaN,NaN,898.0,898.0,898.0,898.0,898.0,898.0
4,617.0,617.0,617.0,617.0,NaN,617.0,NaN,617.0,617.0,617.0


In [390]:
# 3️⃣ Clients × Loss
df_loss = df.pivot(index="client_id", columns="server_round_number", values="round_loss")
df_loss.head()

server_round_number,1,2,3,4,5,6,7,8,9,10
client_id,,,,,,,,,,
0,0.536618,0.404231,0.297476,0.276636,0.269998,0.264570,0.241023,0.268542,0.269131,0.221413
1,0.828988,0.487169,0.292326,NaN,0.261384,NaN,NaN,0.241854,0.237090,0.277091
2,0.894114,0.605754,0.346608,NaN,0.295848,0.283803,0.237611,0.358951,NaN,0.301941
3,NaN,0.419007,NaN,NaN,0.233269,0.193713,0.274899,0.260645,0.190046,0.243337
4,0.863409,0.672737,0.430902,0.391256,NaN,0.326518,NaN,0.288202,0.336713,0.366361


#### Client Specific Statistics

In [391]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_client_metrics(client_id, df_duration, df_loss, df_examples):
    """
    Plot round duration, loss, and examples for a specific client
    
    Parameters:
    client_id (str/int): The client ID to plot
    df_duration (DataFrame): DataFrame with round durations
    df_loss (DataFrame): DataFrame with round losses  
    df_examples (DataFrame): DataFrame with round examples
    """
    
    # Convert client_id to string for consistent matching
    client_id_str = client_id
    
    # Check if client exists in all dataframes
    if client_id_str not in df_duration.index:
        print(f"Client {client_id} not found in duration data")
        return
    if client_id_str not in df_loss.index:
        print(f"Client {client_id} not found in loss data")
        return
    if client_id_str not in df_examples.index:
        print(f"Client {client_id} not found in examples data")
        return
    
    # Get data for the specific client
    client_duration = df_duration.loc[client_id_str]
    client_loss = df_loss.loc[client_id_str]
    client_examples = df_examples.loc[client_id_str]
    
    # Create DataFrames for plotting
    rounds = client_duration.index.tolist()
    
    client_data = pd.DataFrame({
        'Round': rounds,
        'Duration': client_duration.values,
        'Loss': client_loss.values,
        'Examples': client_examples.values
    })
    
    # Create subplot with secondary y-axis
    fig = make_subplots(
        specs=[[{"secondary_y": True}]],
        subplot_titles=[f"Client {client_id} - Round Metrics"]
    )
    
    # Add duration bars
    fig.add_trace(
        go.Bar(
            x=client_data['Round'],
            y=client_data['Duration'],
            name='Duration',
            marker_color='lightblue',
            hovertemplate=(
                "Round: %{x}<br>"
                "Duration: %{y:.4f}s<br>"
                "Loss: %{customdata[0]:.4f}<br>"
                "Examples: %{customdata[1]:.0f}<extra></extra>"
            ),
            customdata=client_data[['Loss', 'Examples']],
            text=client_data['Duration'].round(4),
            textposition='outside',
            texttemplate='%{text:.3f}s',
            showlegend=True
        ),
        secondary_y=False,
    )
    
    # Add loss line
    fig.add_trace(
        go.Scatter(
            x=client_data['Round'],
            y=client_data['Loss'],
            mode='lines+markers+text',
            name='Loss',
            line=dict(color='red', width=3),
            marker=dict(size=8, symbol='diamond'),
            text=client_data['Loss'].round(4),
            textposition='top center',
            texttemplate='%{text:.3f}',
            hovertemplate=(
                "Round: %{x}<br>"
                "Loss: %{y:.4f}<br>"
                "Duration: %{customdata[0]:.4f}s<br>"
                "Examples: %{customdata[1]:.0f}<extra></extra>"
            ),
            customdata=client_data[['Duration', 'Examples']],
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    # Add examples line  
    fig.add_trace(
        go.Scatter(
            x=client_data['Round'],
            y=client_data['Examples'],
            mode='lines+markers+text',
            name='Examples',
            line=dict(color='green', width=3, dash='dot'),
            marker=dict(size=8, symbol='circle'),
            text=client_data['Examples'].round(0),
            textposition='bottom center',
            texttemplate='%{text:.0f}',
            hovertemplate=(
                "Round: %{x}<br>"
                "Examples: %{y:.0f}<br>"
                "Duration: %{customdata[0]:.4f}s<br>"
                "Loss: %{customdata[1]:.4f}<extra></extra>"
            ),
            customdata=client_data[['Duration', 'Loss']],
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    # Update layout
    fig.update_layout(
        title=f"Client {client_id} - Round Duration, Loss and Examples",
        xaxis_title="Rounds",
        yaxis_title="Round Duration (s)",
        yaxis2_title="Loss / Number of Examples",
        legend_title="Metrics",
        template="plotly_white",
        hovermode="closest",
        font=dict(size=12),
        bargap=0.3,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        height=600
    )
    
    # Update y-axes
    fig.update_yaxes(
        title_text="Round Duration (s)",
        secondary_y=False
    )
    
    fig.update_yaxes(
        title_text="Loss / Number of Examples", 
        secondary_y=True
    )
    
    # Update x-axis to show all rounds clearly
    fig.update_xaxes(tickangle=45)
    
    fig.show()
    
    # Return summary statistics
    summary = {
        'client_id': client_id,
        'avg_duration': client_duration.mean(),
        'avg_loss': client_loss.mean(),
        'total_examples': client_examples.iloc[0],  # Since examples are constant per client
        'rounds_analyzed': len(rounds)
    }
    
    print(f"\nSummary for Client {client_id}:")
    print(f"Average Duration: {summary['avg_duration']:.4f}s")
    print(f"Average Loss: {summary['avg_loss']:.4f}")
    print(f"Total Examples: {summary['total_examples']:.0f}")
    print(f"Rounds Analyzed: {summary['rounds_analyzed']}")
    
    return fig, summary

In [392]:
# Plot for a specific client
fig, summary = plot_client_metrics(20, df_duration, df_loss, df_examples)


Summary for Client 20:
Average Duration: 0.3743s
Average Loss: 0.3937
Total Examples: 756
Rounds Analyzed: 10


**** Making sure that we're only processing clients that participated in all rounds ****

In [393]:
# Ensure all dataframes share the same clients first
common_clients = set(df_duration.index) & set(df_examples.index) & set(df_loss.index)

# Align them and drop NaNs
df_duration = df_duration.loc[sorted(common_clients)].dropna(how="any")
df_examples = df_examples.loc[sorted(common_clients)].dropna(how="any")
df_loss = df_loss.loc[sorted(common_clients)].dropna(how="any")

# Now ensure all have the same clients after dropping
common_clients_final = set(df_duration.index) & set(df_examples.index) & set(df_loss.index)
df_duration = df_duration.loc[sorted(common_clients_final)]
df_examples = df_examples.loc[sorted(common_clients_final)]
df_loss = df_loss.loc[sorted(common_clients_final)]

#### COMPREHENSIVE DATAFRAME SUMMARY STATISTICS

In [394]:
# Basic information about each dataframe
print("\n📊 BASIC DATAFRAME INFORMATION")
print("-" * 50)

for name, df in [("Duration", df_duration), ("Loss", df_loss), ("Examples", df_examples)]:
    print(f"\n{name} DataFrame:")
    print(f"  Shape: {df.shape} (clients: {df.shape[0]}, rounds: {df.shape[1]})")
    print(f"  Total values: {df.size}")
    print(f"  Missing values: {df.isnull().sum().sum()}")
    print(f"  Data types: {df.dtypes.unique()}")
    print(f"  Index type: {type(df.index)}")


📊 BASIC DATAFRAME INFORMATION
--------------------------------------------------

Duration DataFrame:
  Shape: (6, 10) (clients: 6, rounds: 10)
  Total values: 60
  Missing values: 0
  Data types: [dtype('float64')]
  Index type: <class 'pandas.core.indexes.base.Index'>

Loss DataFrame:
  Shape: (6, 10) (clients: 6, rounds: 10)
  Total values: 60
  Missing values: 0
  Data types: [dtype('float64')]
  Index type: <class 'pandas.core.indexes.base.Index'>

Examples DataFrame:
  Shape: (6, 10) (clients: 6, rounds: 10)
  Total values: 60
  Missing values: 0
  Data types: [dtype('float64')]
  Index type: <class 'pandas.core.indexes.base.Index'>


#### Summary statistics per round

In [395]:
print("\n\n📈 SUMMARY STATISTICS PER ROUND")
print("-" * 50)

# Duration statistics
print("\n⏱️  ROUND DURATION (seconds):")
print("-" * 30)
duration_stats = df_duration.describe()
print(duration_stats.round(4))

print(f"\nTotal duration across all clients and rounds: {df_duration.sum().sum():.2f}s")
print(f"Average duration per round: {df_duration.mean().mean():.4f}s ± {df_duration.std().mean():.4f}s")

# Loss statistics
print("\n📉 ROUND LOSS:")
print("-" * 30)
loss_stats = df_loss.describe()
print(loss_stats.round(4))

print(f"\nAverage loss per round: {df_loss.mean().mean():.4f} ± {df_loss.std().mean():.4f}")
print(f"Minimum loss: {df_loss.min().min():.4f}")
print(f"Maximum loss: {df_loss.max().max():.4f}")

# Examples statistics
print("\n🔢 NUMBER OF EXAMPLES:")
print("-" * 30)
examples_stats = df_examples.describe()
print(examples_stats.round(2))

print(f"\nTotal examples across all clients and rounds: {df_examples.sum().sum():,.0f}")
print(f"Average examples per client: {df_examples.mean(axis=1).mean():.0f} ± {df_examples.mean(axis=1).std():.0f}")



📈 SUMMARY STATISTICS PER ROUND
--------------------------------------------------

⏱️  ROUND DURATION (seconds):
------------------------------
server_round_number      1       2       3       4       5       6       7   \
count                6.0000  6.0000  6.0000  6.0000  6.0000  6.0000  6.0000   
mean                 0.4700  0.4167  0.4433  0.4383  0.4133  0.4333  0.4267   
std                  0.1049  0.0609  0.0631  0.0794  0.0565  0.0686  0.0599   
min                  0.3500  0.3600  0.3700  0.3600  0.3600  0.3600  0.3600   
25%                  0.3850  0.3700  0.4050  0.3700  0.3700  0.3725  0.4000   
50%                  0.4700  0.4050  0.4350  0.4250  0.4050  0.4300  0.4050   
75%                  0.5325  0.4400  0.4650  0.4950  0.4325  0.4875  0.4475   
max                  0.6200  0.5200  0.5500  0.5500  0.5100  0.5200  0.5300   

server_round_number      8       9       10  
count                6.0000  6.0000  6.0000  
mean                 0.4233  0.4500  0.4267  
std 

#### Client-level Statistics

In [396]:
print("\n\n👥 CLIENT-LEVEL STATISTICS")
print("-" * 50)

# Duration by client
print("\n⏱️  DURATION BY CLIENT (average across rounds):")
client_duration_avg = df_duration.mean(axis=1)
print(f"Fastest client: Client {client_duration_avg.idxmin()} ({client_duration_avg.min():.4f}s)")
print(f"Slowest client: Client {client_duration_avg.idxmax()} ({client_duration_avg.max():.4f}s)")
print(f"Average across clients: {client_duration_avg.mean():.4f}s ± {client_duration_avg.std():.4f}s")

# Loss by client
print("\n📉 LOSS BY CLIENT (average across rounds):")
client_loss_avg = df_loss.mean(axis=1)
print(f"Best performing client: Client {client_loss_avg.idxmin()} ({client_loss_avg.min():.4f})")
print(f"Worst performing client: Client {client_loss_avg.idxmax()} ({client_loss_avg.max():.4f})")
print(f"Average across clients: {client_loss_avg.mean():.4f} ± {client_loss_avg.std():.4f}")



👥 CLIENT-LEVEL STATISTICS
--------------------------------------------------

⏱️  DURATION BY CLIENT (average across rounds):
Fastest client: Client 12 (0.3720s)
Slowest client: Client 0 (0.5370s)
Average across clients: 0.4342s ± 0.0596s

📉 LOSS BY CLIENT (average across rounds):
Best performing client: Client 10 (0.2676)
Worst performing client: Client 19 (0.4330)
Average across clients: 0.3341 ± 0.0641


#### Round-level Statistics

In [397]:
print("\n\n🔄 ROUND-LEVEL STATISTICS")
print("-" * 50)

# Duration by round
print("\n⏱️  DURATION BY ROUND:")
round_duration_avg = df_duration.mean()
fastest_round = round_duration_avg.idxmin()
slowest_round = round_duration_avg.idxmax()
print(f"Fastest round: {fastest_round} ({round_duration_avg.min():.4f}s)")
print(f"Slowest round: {slowest_round} ({round_duration_avg.max():.4f}s)")

# Loss by round
print("\n📉 LOSS BY ROUND:")
round_loss_avg = df_loss.mean()
best_round = round_loss_avg.idxmin()
worst_round = round_loss_avg.idxmax()
print(f"Best round: {best_round} ({round_loss_avg.min():.4f})")
print(f"Worst round: {worst_round} ({round_loss_avg.max():.4f})")

# Examples consistency check
print("\n\n✅ DATA QUALITY CHECKS")
print("-" * 50)

# Check if examples are consistent per client across rounds
examples_consistent = True
for client_id in df_examples.index:
    if df_examples.loc[client_id].nunique() > 1:
        examples_consistent = False
        print(f"⚠️  Client {client_id} has varying examples across rounds")
        break

if examples_consistent:
    print("✓ Examples are consistent for each client across all rounds")
else:
    print("⚠️  Examples vary for some clients across rounds")

# Check for any missing values
missing_duration = df_duration.isnull().sum().sum()
missing_loss = df_loss.isnull().sum().sum()
missing_examples = df_examples.isnull().sum().sum()

if missing_duration == 0 and missing_loss == 0 and missing_examples == 0:
    print("✓ No missing values in any dataframe")
else:
    print(f"⚠️  Missing values - Duration: {missing_duration}, Loss: {missing_loss}, Examples: {missing_examples}")




🔄 ROUND-LEVEL STATISTICS
--------------------------------------------------

⏱️  DURATION BY ROUND:
Fastest round: 5 (0.4133s)
Slowest round: 1 (0.4700s)

📉 LOSS BY ROUND:
Best round: 6 (0.2563)
Worst round: 1 (0.6655)


✅ DATA QUALITY CHECKS
--------------------------------------------------
✓ Examples are consistent for each client across all rounds
✓ No missing values in any dataframe


#### Final summary

In [398]:
print("\n\n🎯 KEY INSIGHTS")
print("-" * 50)
print(f"• Total clients analyzed: {len(df_duration)}")
print(f"• Total rounds completed: {len(df_duration.columns)}")
print(f"• Average round duration: {df_duration.mean().mean():.4f}s")
print(f"• Average loss across all rounds: {df_loss.mean().mean():.4f}")
print(f"• Total training examples: {df_examples.sum().sum():,}")
print(f"• Average examples per client: {df_examples.mean(axis=1).mean():.0f}")



🎯 KEY INSIGHTS
--------------------------------------------------
• Total clients analyzed: 6
• Total rounds completed: 10
• Average round duration: 0.4342s
• Average loss across all rounds: 0.3341
• Total training examples: 54,430.0
• Average examples per client: 907


#### Statistics with Visualizations

##### Training Overview

In [399]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Precompute statistics
client_avg_duration = df_duration.mean(axis=1)
client_avg_loss = df_loss.mean(axis=1)
client_examples = df_examples.iloc[:, 0]  # Examples are consistent across rounds

round_avg_duration = df_duration.mean()
round_avg_loss = df_loss.mean()

fig_overview = go.Figure()

# Key metrics boxes
metrics_fig = go.Figure()

metrics_fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_duration),
    title={"text": "Clients<br><span style='font-size:0.8em;color:gray'>Participating</span>"},
    domain={'row': 0, 'column': 0}
))

metrics_fig.add_trace(go.Indicator(
    mode="number",
    value=len(df_duration.columns),
    title={"text": "Rounds<br><span style='font-size:0.8em;color:gray'>Completed</span>"},
    domain={'row': 0, 'column': 1}
))

metrics_fig.add_trace(go.Indicator(
    mode="number",
    value=df_examples.sum().sum(),
    number={'valueformat': ","},
    title={"text": "Examples<br><span style='font-size:0.8em;color:gray'>Processed</span>"},
    domain={'row': 0, 'column': 2}
))

metrics_fig.add_trace(go.Indicator(
    mode="number",
    value=df_duration.mean().mean(),
    number={'valueformat': ".3f"},
    title={"text": "Avg Duration<br><span style='font-size:0.8em;color:gray'>Seconds</span>"},
    domain={'row': 0, 'column': 3}
))

metrics_fig.update_layout(
    grid={'rows': 1, 'columns': 4, 'pattern': "independent"},
    template="plotly_white",
    height=200
)

metrics_fig.show()


##### Client Performance Distribution

In [400]:
fig_clients = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Average Training Duration by Client',
        'Average Loss by Client', 
        'Training Examples per Client',
        'Duration vs Loss Correlation'
    ),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]]
)

# Duration distribution
fig_clients.add_trace(
    go.Bar(
        x=client_avg_duration.index,
        y=client_avg_duration.values,
        name='Avg Duration',
        marker_color='lightblue',
        hovertemplate="Client: %{x}<br>Avg Duration: %{y:.3f}s<extra></extra>"
    ),
    row=1, col=1
)

# Loss distribution
fig_clients.add_trace(
    go.Bar(
        x=client_avg_loss.index,
        y=client_avg_loss.values,
        name='Avg Loss',
        marker_color='lightcoral',
        hovertemplate="Client: %{x}<br>Avg Loss: %{y:.3f}<extra></extra>"
    ),
    row=1, col=2
)

# Examples distribution
fig_clients.add_trace(
    go.Bar(
        x=client_examples.index,
        y=client_examples.values,
        name='Examples',
        marker_color='lightgreen',
        hovertemplate="Client: %{x}<br>Examples: %{y:.0f}<extra></extra>"
    ),
    row=2, col=1
)

# Duration vs Loss scatter
fig_clients.add_trace(
    go.Scatter(
        x=client_avg_duration.values,
        y=client_avg_loss.values,
        mode='markers',
        marker=dict(
            size=10,
            color=client_examples.values,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Examples")
        ),
        text=client_avg_duration.index,
        hovertemplate="Client: %{text}<br>Duration: %{x:.3f}s<br>Loss: %{y:.3f}<br>Examples: %{marker.color:.0f}<extra></extra>",
        name='Client Performance'
    ),
    row=2, col=2
)

fig_clients.update_layout(
    height=800,
    showlegend=False,
    template="plotly_white",
    title_text="Client Performance Analysis"
)

fig_clients.update_xaxes(title_text="Client ID", row=1, col=1)
fig_clients.update_xaxes(title_text="Client ID", row=1, col=2)
fig_clients.update_xaxes(title_text="Client ID", row=2, col=1)
fig_clients.update_xaxes(title_text="Duration (s)", row=2, col=2)

fig_clients.update_yaxes(title_text="Duration (s)", row=1, col=1)
fig_clients.update_yaxes(title_text="Loss", row=1, col=2)
fig_clients.update_yaxes(title_text="Examples", row=2, col=1)
fig_clients.update_yaxes(title_text="Loss", row=2, col=2)

fig_clients.show()


##### Round Progress Over Time

In [401]:
fig_rounds = make_subplots(
    specs=[[{"secondary_y": True}]]
)

# Duration trend
fig_rounds.add_trace(
    go.Scatter(
        x=round_avg_duration.index,
        y=round_avg_duration.values,
        mode='lines+markers',
        name='Avg Duration',
        line=dict(color='blue', width=3),
        marker=dict(size=8),
        hovertemplate="Round: %{x}<br>Avg Duration: %{y:.3f}s<extra></extra>"
    ),
    secondary_y=False
)

# Loss trend
fig_rounds.add_trace(
    go.Scatter(
        x=round_avg_loss.index,
        y=round_avg_loss.values,
        mode='lines+markers',
        name='Avg Loss',
        line=dict(color='red', width=3),
        marker=dict(size=8, symbol='diamond'),
        hovertemplate="Round: %{x}<br>Avg Loss: %{y:.3f}<extra></extra>"
    ),
    secondary_y=True
)

fig_rounds.update_layout(
    title="Training Progress Across Rounds",
    xaxis_title="Round Number",
    template="plotly_white",
    height=500
)

fig_rounds.update_yaxes(title_text="Average Duration (s)", secondary_y=False)
fig_rounds.update_yaxes(title_text="Average Loss", secondary_y=True)

fig_rounds.show()


##### Performance Rankings

In [402]:
# Fastest clients
fastest_clients = client_avg_duration.nsmallest(5)
# Best performing clients (lowest loss)
best_clients = client_avg_loss.nsmallest(5)

fig_performers = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Fastest Clients (Lowest Duration)', 'Best Performing Clients (Lowest Loss)')
)

fig_performers.add_trace(
    go.Bar(
        x=fastest_clients.values,
        y=fastest_clients.index,
        orientation='h',
        marker_color='green',
        name='Fastest',
        hovertemplate="Client: %{y}<br>Duration: %{x:.3f}s<extra></extra>"
    ),
    row=1, col=1
)

fig_performers.add_trace(
    go.Bar(
        x=best_clients.values,
        y=best_clients.index,
        orientation='h',
        marker_color='orange',
        name='Best Performance',
        hovertemplate="Client: %{y}<br>Loss: %{x:.3f}<extra></extra>"
    ),
    row=1, col=2
)

fig_performers.update_layout(
    height=400,
    template="plotly_white",
    showlegend=False
)

fig_performers.update_xaxes(title_text="Duration (s)", row=1, col=1)
fig_performers.update_xaxes(title_text="Loss", row=1, col=2)

fig_performers.show()


##### Key Insights

In [403]:
total_training_time = df_duration.sum().sum()
avg_loss_improvement = (df_loss.iloc[:, 0].mean() - df_loss.iloc[:, -1].mean()) / df_loss.iloc[:, 0].mean() * 100

insights = [
    f"• 🎯 {len(df_duration)} clients participated in {len(df_duration.columns)} training rounds",
    f"• ⏱️  Total training time: {total_training_time:.1f} seconds ({total_training_time/60:.1f} minutes)",
    f"• 📊 Processed {df_examples.sum().sum():,} training examples in total",
    f"• 📉 Model improved by {avg_loss_improvement:.1f}% from first to last round",
    f"• 🏆 Fastest client: Client {client_avg_duration.idxmin()} ({client_avg_duration.min():.3f}s per round)",
    f"• 🎯 Best performer: Client {client_avg_loss.idxmin()} ({client_avg_loss.min():.3f} average loss)",
    f"• ⚡ Average round duration: {df_duration.mean().mean():.3f} seconds",
    f"• 📈 Training shows {'positive' if avg_loss_improvement > 0 else 'negative'} progress across rounds"
]

for insight in insights:
    print(insight)

• 🎯 6 clients participated in 10 training rounds
• ⏱️  Total training time: 26.0 seconds (0.4 minutes)
• 📊 Processed 54,430.0 training examples in total
• 📉 Model improved by 60.2% from first to last round
• 🏆 Fastest client: Client 12 (0.372s per round)
• 🎯 Best performer: Client 10 (0.268 average loss)
• ⚡ Average round duration: 0.434 seconds
• 📈 Training shows positive progress across rounds


In [404]:
print("✅ Clients with complete data across all rounds:")
print(df_duration.index.tolist())

✅ Clients with complete data across all rounds:
[0, 10, 12, 14, 19, 46]


In [405]:
import matplotlib.pyplot as plt

# Ensure plots are large and readable
plt.rcParams["figure.figsize"] = (10, 6)

#### 🎯 Purpose of This Visualization

This visualization provides an interactive overview of how different clients perform across multiple training rounds in a federated learning setup. It highlights three key metrics:  

- **Round Duration** — how long each client took per round.  
- **Loss** — how well each client’s model performed on average.  
- **Number of Examples** — how much data each client contributed.  

By combining bars and lines on dual axes, the plot helps identify performance trends, imbalances, and correlations between computation time, model accuracy, and data volume across clients.  

In short, it offers a clear, comparative view of **client efficiency, reliability, and contribution** throughout the training process.


In [406]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sort and convert client IDs to strings for categorical spacing
df_duration = df_duration.sort_index()
df_duration.index = df_duration.index.astype(str)
df_loss = df_loss.sort_index()
df_loss.index = df_loss.index.astype(str)
df_examples = df_examples.sort_index()
df_examples.index = df_examples.index.astype(str)

# Melt all DataFrames
df_duration_melted = df_duration.reset_index().melt(
    id_vars="client_id",
    var_name="Round",
    value_name="Duration"
)

df_loss_melted = df_loss.reset_index().melt(
    id_vars="client_id", 
    var_name="Round",
    value_name="Loss"
)

df_examples_melted = df_examples.reset_index().melt(
    id_vars="client_id",
    var_name="Round", 
    value_name="Examples"
)

# Merge all data
df_merged = df_duration_melted.merge(
    df_loss_melted, on=["client_id", "Round"]
).merge(
    df_examples_melted, on=["client_id", "Round"]
)

# Create subplot with secondary y-axis
fig = make_subplots(
    specs=[[{"secondary_y": True}]]
)

# Get unique rounds for coloring
rounds = df_merged['Round'].unique()
colors = px.colors.qualitative.Set3

# Add bars for each round
for i, round_name in enumerate(rounds):
    round_data = df_merged[df_merged['Round'] == round_name]
    
    fig.add_trace(
        go.Bar(
            x=round_data['client_id'],
            y=round_data['Duration'],
            name=f"{round_name}",
            legendgroup=round_name,
            marker_color=colors[i % len(colors)],
            hovertemplate=(
                "Client: %{x}<br>"
                f"Round: {round_name}<br>"
                "Duration: %{y:.4f}s<br>"
                "Loss: %{customdata[0]:.4f}<br>"
                "Examples: %{customdata[1]:.0f}<extra></extra>"
            ),
            customdata=round_data[['Loss', 'Examples']],
            text=round_data['Duration'].round(4),
            textposition='outside',
            texttemplate='%{text:.3f}s',
            showlegend=True
        ),
        secondary_y=False,
    )

# Add loss line
fig.add_trace(
    go.Scatter(
        x=df_merged['client_id'].unique(),
        y=df_loss.mean(axis=1),  # Average loss per client across rounds
        mode='lines+markers',
        name='Avg Loss',
        line=dict(color='red', width=3, dash='dot'),
        marker=dict(size=8, symbol='diamond'),
        hovertemplate=(
            "Client: %{x}<br>"
            "Average Loss: %{y:.4f}<extra></extra>"
        ),
        yaxis='y2'
    ),
    secondary_y=True,
)

# Add examples line  
fig.add_trace(
    go.Scatter(
        x=df_merged['client_id'].unique(),
        y=df_examples.mean(axis=1),  # Average examples per client across rounds
        mode='lines+markers',
        name='Avg Examples',
        line=dict(color='blue', width=3),
        marker=dict(size=8, symbol='circle'),
        hovertemplate=(
            "Client: %{x}<br>"
            "Average Examples: %{y:.0f}<extra></extra>"
        ),
        yaxis='y2'
    ),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title="Clients Metrics (The Loss, and Duration) for each round with respect to the Number of Examples",
    xaxis_title="Client ID",
    yaxis_title="Round Duration (seconds)",
    yaxis2_title="Loss / Number of Examples",
    legend_title="Metrics & Rounds",
    template="plotly_white",
    hovermode="closest",
    font=dict(size=10),
    bargap=0.2,
    bargroupgap=0.05,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Update y-axes
fig.update_yaxes(
    title_text="Round Duration (s)",
    secondary_y=False
)

fig.update_yaxes(
    title_text="Loss / Number of Examples", 
    secondary_y=True,
    # Optional: if you want to separate the scales more clearly
    # range=[0, max(df_examples.max().max(), df_loss.max().max()) * 1.1]
)

fig.show()

END OF NOTEBOOK